<a href="https://colab.research.google.com/github/KNL1979/DS807/blob/main/Q3_CNN_compressed_img_from_VAE4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN classification using compressed images from VAE4

In [ ]:
# U=upgrade and q=quiet
!pip install wandb -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd '/content/drive/My Drive/Applied_ML'

/content/drive/My Drive/Applied_ML


In [ ]:
import os
import wandb
from wandb.keras import WandbCallback

import pprint

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, Flatten, Dropout,BatchNormalization, SpatialDropout2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np

In [ ]:
# load compressed images (z vector) from VAE4
# The x_train is in many subsets because of RAM limitations and other computer issues during extraction of the compressed z data.
X_train1 = np.load('eksamen/VAE4/Z_Xtrain1_VAE4.npy')
X_train2 = np.load('eksamen/VAE4/Z_Xtrain2_VAE4.npy')
X_train3 = np.load('eksamen/VAE4/Z_Xtrain3_VAE4.npy')
X_train4 = np.load('eksamen/VAE4/Z_Xtrain4_VAE4.npy')
X_train5 = np.load('eksamen/VAE4/Z_Xtrain5_VAE4.npy')
X_train6 = np.load('eksamen/VAE4/Z_Xtrain6_VAE4.npy')
# X_train7 = np.load('C:/Users/mette/OneDrive/SDU\semester_3/Applied_mashine_learning/Eksamen/VAE4_z_dat/asetZ_Xtrain7_VAE4.npy') dette interval er downloaded 2x ved en fejl
X_train8 = np.load('eksamen/VAE4/Z_Xtrain8_VAE4.npy')
X_train9 = np.load('eksamen/VAE4/Z_Xtrain9_VAE4.npy')

X_val = np.load('eksamen/VAE4/Z_Xval_VAE4.npy')

X_test = np.load('eksamen/VAE4/Z_Xtest_VAE4.npy')

In [ ]:
print(X_train1.shape, X_train2.shape, X_train3.shape, X_train4.shape, X_train5.shape,X_train6.shape, X_train8.shape, X_train9.shape, X_val.shape, X_test.shape)

(30000, 1024) (5000, 1024) (5000, 1024) (3000, 1024) (3000, 1024) (5000, 1024) (3000, 1024) (1050, 1024) (15729, 1024) (7864, 1024)


In [ ]:
X_train = np.concatenate((X_train1, X_train2, X_train3, X_train4, X_train5, X_train6, X_train8, X_train9), axis=0)
print(X_train.shape)

(55050, 1024)


In [ ]:
# load label
y_train = np.load('eksamen/dataset/y_train_new.npy')
y_val = np.load('eksamen/dataset/y_val_new.npy')
y_test = np.load('eksamen/dataset/y_test_new.npy')
print((y_train.shape, y_val.shape, y_test.shape))

((55050, 2), (15729, 2), (7864, 2))


In [ ]:
# merge train and validation datasets
X_train_new = np.concatenate([X_train, X_val])
y_train_new = np.concatenate([y_train, y_val])
print(X_train_new.shape, y_train_new.shape)

(70779, 1024) (70779, 2)


In [ ]:
# settings for Wandb
os.environ["WANDB_API_KEY"]= '1bff76125a134665e9d30342c447d7616c5fd3c3'
os.environ["WANDB_NAME"]= 'VAE4_CNN3_BEST'
os.environ["WANDB_NOTEBOOK_NAME"]='VAE4_CNN3_BEST.ipynb'
run = wandb.init(project="VAE1_CNN",entity='coffeegang',job_type='train')
config = wandb.config

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find VAE4_CNN3_BEST.ipynb.
wandb: Currently logged in as: metterosenfjeld (coffeegang). Use `wandb login --relogin` to force relogin


In [ ]:
# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(monitor='val_loss',verbose=True,save_model=False,
                               log_weights=True,
                               log_evaluation=True,
                               validation_steps=2)
callbacks = [wandb_callback]

In [ ]:
# define model
model3 = tf.keras.models.Sequential([
    Dense(units=6*6*512, activation='relu', input_shape=(1024,)),
    Reshape(target_shape=(6, 6, 512)), # To get in "image format"
    Conv2DTranspose(512, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2DTranspose(256, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2DTranspose(128, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2DTranspose(64, 3, 2, padding='same', activation='relu'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2DTranspose(3, 3, 1, padding='same'), # The decoder architecture stops here
    Conv2D(64, 3, 2, activation='relu', padding='same'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Conv2D(128, 3, 2, activation='relu', padding='same'),
    BatchNormalization(),
    SpatialDropout2D(0.2),
    Flatten(),
    Dense(500, activation='relu'),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18432)             18892800  
                                                                 
 reshape (Reshape)           (None, 6, 6, 512)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 12, 12, 512)       2359808   
 anspose)                                                        
                                                                 
 batch_normalization (Batch  (None, 12, 12, 512)       2048      
 Normalization)                                                  
                                                                 
 spatial_dropout2d (Spatial  (None, 12, 12, 512)       0         
 Dropout2D)                                                      
                                                        

In [ ]:
# compile model
model3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )

In [ ]:
# fit model
history3 = model3.fit(X_train_new, y_train_new, epochs=15, verbose=1, callbacks = callbacks)

wandb: WARNING WandbCallback is unable to read validation_data from trainer and therefore cannot log validation data. Ensure Keras is properly patched by calling `from wandb.keras import WandbCallback` at the top of your script.


Epoch 1/15
   5/2212 [..............................] - ETA: 1:17 - loss: 6.2794 - accuracy: 0.6187

2212/2212 [==============================] - 87s 35ms/step - loss: 0.6155 - accuracy: 0.6873
Epoch 2/15
2212/2212 [==============================] - 76s 35ms/step - loss: 0.5012 - accuracy: 0.7530
Epoch 3/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4834 - accuracy: 0.7675
Epoch 4/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4712 - accuracy: 0.7718
Epoch 5/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4604 - accuracy: 0.7781
Epoch 6/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4540 - accuracy: 0.7821
Epoch 7/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4446 - accuracy: 0.7848
Epoch 8/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4366 - accuracy: 0.7895
Epoch 9/15
2212/2212 [==============================] - 76s 34ms/step - loss: 0.4286 - accuracy: 0.7933
Epoch 10/15
2212/2212 [==============================] - 76s 34ms/step - lo

In [ ]:
wandb.finish()

accuracy,▁▄▅▅▅▆▆▆▆▆▇▇▇██
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▄▄▄▃▃▃▃▂▂▂▁▁
accuracy,0.8308
epoch,14
loss,0.36325


In [ ]:
# evaluate test dataset
result = model3.evaluate(X_test,y_test)

246/246 [==============================] - 3s 11ms/step - loss: 0.4703 - accuracy: 0.7968
